In [56]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Read data

In [83]:
data = {"january": "data/yellow_tripdata_2023-01.parquet", "february": "data/yellow_tripdata_2023-02.parquet"}
jan_data = pd.read_parquet(data['january'])

# Q1: Read the data for January. How many columns are there?

In [84]:
print('Number of columns are: ', len(jan_data.columns))

Number of columns are:  19


# Q2: What's the standard deviation of the trips duration in January?

## preprocess data

In [85]:
# Check if all values in tpep_dropoff_datetime are greater than tpep_pickup_datetime
all_dropoff_after_pickup = (jan_data['tpep_dropoff_datetime'] > jan_data['tpep_pickup_datetime']).all()

if all_dropoff_after_pickup:
    print("All tpep_dropoff_datetime values are after tpep_pickup_datetime.")
else:
    print("There are some rows where tpep_dropoff_datetime is not after tpep_pickup_datetime.")

There are some rows where tpep_dropoff_datetime is not after tpep_pickup_datetime.


In [86]:
equal_rows = jan_data[jan_data['tpep_dropoff_datetime'] == jan_data['tpep_pickup_datetime']]
invalid_rows = jan_data[jan_data['tpep_dropoff_datetime'] < jan_data['tpep_pickup_datetime']]

In [87]:
print(equal_rows[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].head())
print("\n")
print(invalid_rows[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].head())

      tpep_pickup_datetime tpep_dropoff_datetime
3291   2023-01-01 00:11:55   2023-01-01 00:11:55
4971   2023-01-01 01:54:01   2023-01-01 01:54:01
11648  2023-01-01 02:45:08   2023-01-01 02:45:08
15664  2023-01-01 03:55:38   2023-01-01 03:55:38
22644  2023-01-01 07:24:56   2023-01-01 07:24:56


        tpep_pickup_datetime tpep_dropoff_datetime
1839180  2023-01-20 13:35:00   2023-01-20 13:05:48
2124414  2023-01-23 10:43:58   2023-01-23 10:29:26
2218504  2023-01-24 11:00:00   2023-01-24 10:40:48


## Seen problem: There are some rows where tpep_dropoff_datetime is not after tpep_pickup_datetime.
so let's remove them from data

In [88]:
valid_df = jan_data[jan_data['tpep_dropoff_datetime'] > jan_data['tpep_pickup_datetime']].copy()

In [89]:
valid_df['trip_duration_minute'] = (valid_df['tpep_dropoff_datetime'] - valid_df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [90]:
print("the real standard deviation of the trips duration in January is: ", round(valid_df['trip_duration_minute'].std(), 2))

the real standard deviation of the trips duration in January is:  42.6


# the real standard deviation of the trips duration in January is 42.6, but the answer not exists in question, so let's try ugly way

In [91]:
jan_data['trip_duration_minute'] = (jan_data['tpep_dropoff_datetime'] - jan_data['tpep_pickup_datetime']).dt.total_seconds() / 60
print("standard deviation of the trips duration in January is: ", round(jan_data['trip_duration_minute'].std(), 2))

standard deviation of the trips duration in January is:  42.59


# Q3: What fraction of the records left after you dropped the outliers?

In [92]:
filtered_jan_data = jan_data[(jan_data['trip_duration_minute']>=1) &(jan_data['trip_duration_minute']<=60)].copy()

In [93]:
print("fraction of the records left after we dropped the outliers is: " ,round((len(filtered_jan_data)*100)/len(jan_data), 2))

fraction of the records left after we dropped the outliers is:  98.12


# Q4: What's the dimensionality of this matrix (number of columns)?

In [94]:
filtered_jan_data.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee',
       'trip_duration_minute'],
      dtype='object')

In [95]:
categorical_columns = ['PULocationID', 'DOLocationID']
filtered_jan_data[categorical_columns] = filtered_jan_data[categorical_columns].astype(str)

In [96]:
filtered_jan_data[categorical_columns].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [97]:
train_dict = filtered_jan_data[categorical_columns].to_dict(orient='records').copy()

In [98]:
dict_vectorizer = DictVectorizer()
X_train = dict_vectorizer.fit_transform(train_dict)
print(f"the dimensionality of X_train is: {X_train.shape[1]}")

the dimensionality of X_train is: 515


# Q5: What's the RMSE on train?

In [99]:
y_train = filtered_jan_data['trip_duration_minute'].values

In [100]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [101]:
y_pred  = lr.predict(X_train)

In [102]:
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

RMSE: 7.649261927686161


# Q6: What's the RMSE on validation?

In [62]:
# Read data
feb_data = pd.read_parquet(data['february'])

# Preprocessing
feb_data['trip_duration_minute'] = (feb_data['tpep_dropoff_datetime'] - feb_data['tpep_pickup_datetime']).dt.total_seconds() / 60
filtered_feb_data = feb_data[(feb_data['trip_duration_minute']>=1) &(feb_data['trip_duration_minute']<=60)].copy()
# filtered_feb_data = feb_data.copy()
filtered_feb_data[categorical_columns] = filtered_feb_data[categorical_columns].astype(str)

# One hot encoding
train_dict = filtered_feb_data[categorical_columns].to_dict(orient='records').copy()
dict_vectorizer = DictVectorizer()
X_train = dict_vectorizer.fit_transform(train_dict)
y_train = filtered_feb_data['trip_duration_minute'].values

# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Calculate RMSE
y_pred  = lr.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

RMSE: 7.778948571132272
